In [ ]:
import tensorflow as tf
from minio import Minio
from tensorflow import keras
import os

In [ ]:
# Config Paramters
minio_address = "minio.ns-1.svc.cluster.local"
minio_access_key = "kubeflow"
minio_secret_key = "kubeflow123"
datasets_bucket = "datasets"

In [ ]:
preprocessed_data_folder = "preprocessed-data"
tf_record_file_size = 500


In [ ]:
# Configure TF to use MinIO
os.environ["AWS_ACCESS_KEY_ID"] = minio_access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = minio_secret_key
os.environ["AWS_REGION"] = "us-east-1"
os.environ["S3_ENDPOINT"] = minio_address
os.environ["S3_USE_HTTPS"] = "0"
os.environ["S3_VERIFY_SSL"] = "0"

In [ ]:
minioClient = Minio(minio_address,
                    access_key=minio_access_key,
                    secret_key=minio_secret_key,
                    secure=False)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

In [ ]:
# List all testing tfrecord files
objects = minioClient.list_objects(datasets_bucket, prefix=f"{preprocessed_data_folder}/test")
testing_files_list = []
for obj in objects:
    testing_files_list.append(obj.object_name)

In [ ]:
testing_filenames = [f"s3://datasets/{f}" for f in testing_files_list]

In [ ]:
testing_dataset = tf.data.TFRecordDataset(testing_filenames, num_parallel_reads=AUTO, compression_type="GZIP")
testing_dataset = testing_dataset.with_options(ignore_order)


In [ ]:
def decode_fn(record_bytes):
    schema = {
        "label": tf.io.FixedLenFeature([2], dtype=tf.int64),
        "sentence": tf.io.FixedLenFeature([512], dtype=tf.float32),
    }

    tf_example = tf.io.parse_single_example(record_bytes, schema)
    new_shape = tf.reshape(tf_example["sentence"], [1, 512])
    label = tf.reshape(tf_example["label"], [1, 2])
    return new_shape, label


In [ ]:
testing_mapped_ds = testing_dataset.map(decode_fn)
testing_mapped_ds = testing_mapped_ds.repeat(5)
testing_mapped_ds = testing_mapped_ds.batch(128)

In [ ]:
# load model
model_destination = f"s3://{datasets_bucket}/imdb_sentiment_analysis/1"
model = keras.models.load_model(model_destination)

In [ ]:
testing = model.evaluate(testing_mapped_ds)
print(testing)

with open(output_text_path, 'w') as writer:
    writer.write("done eval!")